In [1]:
import openslide
from matplotlib.colors import ListedColormap
from skimage import color
import skimage.morphology  as skmp

In [2]:
import glob
import tifffile
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time
import skimage.io as io

## class distribution

### train

In [ ]:
df_train = pd.read_csv('/data/neuro/stroke/kaggle_strip_ai/train.csv')
df_train

In [ ]:
print('CE class: ', df_train[df_train['label'] == 'CE'].shape[0])
print('LLA class: ', df_train[df_train['label'] == 'LAA'].shape[0])

### test

In [ ]:
df_test = pd.read_csv('/data/neuro/stroke/kaggle_strip_ai/test.csv')
df_test

## Data extraction and preprocessing

In [ ]:
all_files = glob.glob('/data/neuro/stroke/kaggle_strip_ai/train/*.tif')
all_files

### openslide analysis

In [ ]:
slide_path = all_files[0]
slide = openslide.OpenSlide(slide_path)

In [ ]:
image_id = slide_path.split('/')[::-1][0].split('.')[0]
df_train[df_train['image_id']==str(image_id)]['label']

In [ ]:
print('stored dimenions: ', slide.level_dimensions)
print('stored downsampled levels: ', slide.level_downsamples)
print('slide_properties: ', slide.properties)

### extract thumbnail - lower resolution

In [ ]:
### try exctracting thumbnail using skimage
t1 = time.time()
resize_factor = 8
img = io.imread(slide_path)
thumbnail = cv2.resize(img, dsize=None, fx=1/resize_factor, fy=1/resize_factor)
print('thumbnail shape: ', thumbnail.shape, 'time: ', time.time() - t1)

In [ ]:
### try exctracting thumbnail using skimage and bicubic interpolation
t1 = time.time()
resize_factor = 8
img = io.imread(slide_path)
thumbnail = cv2.resize(img, dsize=None, fx=1/resize_factor, fy=1/resize_factor, interpolation=cv2.INTER_CUBIC)
print('thumbnail shape: ', thumbnail.shape, 'time: ', time.time() - t1)

In [ ]:
### try exctracting thumbnail using openslide
t1 = time.time()
down_rate = 8
size = np.array(slide.level_dimensions)[0]//down_rate
print(size)
thumbnail = np.array(slide.get_thumbnail([size[0], size[1]]).convert('RGB'))
print('thumbnail shape: ', thumbnail.shape, 'time: ', time.time() - t1)

In [ ]:
### try exctracting thumbnail using tifffile
down_rate = 16
tif = tifffile.TiffFile(slide_path)
image  = tif.series[0].asarray().astype(np.uint8)
thumbnail = cv2.resize(image, (tif.series[0].shape[1]//down_rate,tif.series[0].shape[0]//down_rate), interpolation=cv2.INTER_CUBIC)
print('thumbnail shape: ', thumbnail.shape, 'time: ', time.time() - t1)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(thumbnail, cmap='gray')
plt.axis('off')

In [ ]:
from PIL import Image

from histolab.filters.image_filters import (
    ApplyMaskImage,
    Compose,
    OtsuThreshold,
    RgbToGrayscale,GreenPenFilter,BluePenFilter
)
from histolab.filters.morphological_filters import BinaryDilation,BinaryErosion,BinaryClosing,BinaryOpening,RemoveSmallObjects,RemoveSmallHoles

def composed_filters(image_rgb):
    filters = Compose(
        [
            RgbToGrayscale(),
            OtsuThreshold(),
#             BinaryDilation(),
#             BinaryErosion(),
            BinaryClosing(),
            RemoveSmallHoles(),
#             BinaryClosing(),
#             BinaryDilation(),
            RemoveSmallObjects(),
            ApplyMaskImage(image_rgb),
            BluePenFilter(),
            GreenPenFilter(),

        ]
    )
    return filters(image_rgb)


image_rgb = Image.fromarray(thumbnail)
resulting_image = composed_filters(image_rgb)


In [ ]:
np.array(resulting_image).shape

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(resulting_image)
plt.axis('off')

In [ ]:
from histolab.tiler import GridTiler

In [ ]:
from histolab.slide import Slide
slide = Slide(slide_path, processed_path='/data/eredekop/stroke_kaggle/processed')

In [ ]:
plt.imshow(np.array(slide.thumbnail))

In [ ]:
grid_tiles_extractor = GridTiler(
   tile_size=(512, 512),
   level=0,
   check_tissue=True,
   pixel_overlap=0, # default
   prefix="grid/", # save tiles in the "grid" subdirectory of slide's processed_path
   suffix=".png" # default
)

In [ ]:
grid_tiles_extractor.locate_tiles(
    slide=slide,
    scale_factor=64,
    alpha=64,
    outline="#046C4C",
)

In [ ]:
!pip install histolab

In [ ]:
!pip install scikit-image==0.18.1